# 網站設計 Web_Framework_for_Flask

檔案結構   
guessnumlib.py  
執行指令    
pip install flask   
python guessnumlib.py  

## 1.1猜數字遊戲 guessnumlib.py
|記錄|題目|答案|作答|OK檢查|提示|最大次數|試了次數||
|---|---|---|---|---|---|---|---|---|
| TE={| St|Val|Ans|OK|Tip|MaxTry|TryCnt|}|
|   | 描述| 隨機數|  |0/1|太大/太小|10|0|

![GitHub Logo](../img/guessnum.png)


In [ ]:
# guessnumlib.py
import random    
import datetime
def GetKey():
    return r"%s.%s"% (datetime.datetime.now().isoformat(),random.random())

def GetTE( St, Val):
    return { "St": St,"Val": Val,"Ans": "","OK": 0, "Tip": "請輸數字", "MaxTry":10, "TryCnt": 0 }

def Get_Expr(QID="1"):
    if   QID ==  "1":MaxNum=10
    elif QID ==  "2":MaxNum=100
    elif QID ==  "3":MaxNum=1000
    Val= random.choice(range(1,MaxNum))
    St=f"Guess 1-{MaxNum}"
    return GetTE(St,Val)

def Post_Expr_CheckAns(TE):
    Ans=int(TE["Ans"]) if TE["Ans"].isnumeric() else -1
    if TE["Val"]==Ans:
        TE["OK"]=1
        TE["Tip"]="Well done!"
    elif TE["TryCnt"]>TE["MaxTry"]:
        TE["OK"] = -1
        TE["Tip"]="Game over!"
    elif TE["Val"]<Ans:
        TE["Tip"]="Too high, try again!"
    elif TE["Val"]>Ans:
        TE["Tip"]="Too low, try again!"
        
if __name__ == '__main__': #主程式
    TE=Get_Expr()      # TE.Val=隨機數
    print(TE["St"])    # 顯示題目描述 TE.St
    while TE["OK"]==0: # 如果 TE.OK==0 迴圈
        TE["Ans"]=input(TE["Tip"])  # TE.Ans=輸入數字
        TE["TryCnt"]+=1             # TE.TryCnt 增加數字 
        Post_Expr_CheckAns(TE)      # 檢查 TE.Val==TE.Ans, 相等 1 否則 0
    print(TE["Tip"])  # 顯示結果      

## 1.2網站Hello World (guessnum.py)

In [ ]:
from flask import Flask

def create_app():
    app = Flask(__name__)

    # 根路由
    @app.route("/")
    def index():
        return "Hello World!"

    # 容錯處理
    @app.errorhandler(500)
    def server_error(e):
        return """內部錯誤: <pre>{}</pre>,請檢查日誌.""".format(e), 500

    return app

app = create_app()

# the application.
if __name__ == '__main__':
    app.run( host="0.0.0.0",port=8881, debug=True)

## 1.3網遊猜數字遊戲 guessnum.py

In [ ]:
from flask import Flask, redirect, request
import random    
import datetime
import guessnumlib as lib

NTE={}

def html_template( title, TE, SID):  
    return """
    <form method=POST>
    <div>{}</div>
    <div>{}</div>
    <input name=Ans>
    <input type=hidden name=SID value='{}'>
    <div>Try {}</div>
    <input type=submit>
    </form>
    """.format(TE["St"],TE["Tip"],SID,TE["TryCnt"])

def create_app():
    app = Flask(__name__)

    @app.route("/")
    def index():
        return redirect('/apps')

    @app.route("/apps")
    def list():
        books=["1.1-10","2.1-100","3.1-1000"]
        return "<ul>{}</ul>".format("".join(f"<li><a href=apps/{r}>{r}</a>" for r in books))

    @app.route("/apps/<QID>", methods=['GET', 'POST'])
    def GamePanel(QID):
        QIID=QID.split(".")[0]
        if request.method == 'POST':
            SID = request.form["SID"]
            TE=NTE.get(SID,None)
            TE["Ans"]=request.form["Ans"]
            TE["TryCnt"]+=1            
            lib.Post_Expr_CheckAns(TE)          
            if TE["OK"]==0:
                NTE[SID] =TE 
                return html_template(QID,TE,SID)
            else:
                return "<p>{}</p><a href=/>返回</a>".format(TE["Tip"])
            
        SID=lib.GetKey()
        TE=lib.Get_Expr(QIID)
        NTE[SID] =TE        
        return html_template(QID,TE,SID)

    # 容錯處理
    @app.errorhandler(500)
    def server_error(e):
        return """內部錯誤:<pre>{}</pre>,請檢查日誌. <a href=/>返回</a>""".format(e), 500

    return app

app = create_app()

if __name__ == '__main__':
    app.run( host="0.0.0.0",port=8881, debug=True)

## 1.4 模板 template

In [ ]:
from flask import current_app, Flask, redirect, request, url_for,render_template
import random    # 自定義,數學出題庫
import datetime
import guessnumlib as lib
NTE={}

def create_app():
    app = Flask(__name__)

    # 根路由
    @app.route("/")
    def index():
        return redirect('/apps')

    # 使用模版list.html, 顯示列表 
    @app.route("/apps")
    def list():
        books=["PF101.10.1-10","PF102.10.1-100","PF103.10.1-1000"]
        return render_template(
            "list.html",
            books=books
        ) 

    @app.route("/apps/<QID>", methods=['GET', 'POST'])
    def MathPanel(QID):
        QIID=QID.split(".")[0]
        if request.method == 'POST':
            SID = request.form["SID"]
            TE=NTE.get(SID,None)
            TE["TryCnt"]+=1
            TE["Ans"]=request.form["Ans"]
            lib.Post_Expr_CheckAns(TE)          
            if TE["OK"]==0:
                NTE[SID] =TE 
                return render_template("guessnum.html", title=QID, TE=TE, SID=SID)
            else:
                NTE.pop(SID,None)
                return "<p>{}</p><a href=/>返回</a>".format(TE["Tip"])

        TE=lib.Get_Expr(QIID)
        SID=lib.GetKey()
        NTE[SID] =TE        
        return render_template("guessnum.html", title=QID, TE=TE, SID=SID)                                

    # 容錯處理
    @app.errorhandler(500)
    def server_error(e):
        return """內部錯誤: <pre>{}</pre>,請檢查日誌.""".format(e), 500

    return app

app = create_app()

# the application.
if __name__ == '__main__':
    app.run( host="0.0.0.0",port=8881, debug=True)

## 1.5 flask_session 機制
python -m venv env     
env\scripts\activate  
pip install -r requirements.txt  
python main.py  
env\scripts\deactivate  
requirements.txt  

```
Flask==1.1.2
Flask-Session==0.3.2
six==1.15.0
httplib2==0.18.1
sympy==1.7.1
```
檔案結構   
guessnumlib.py  
guessnum.py  
templates\base.html   
templates\list.html  
templates\guessnum.html 

In [ ]:
from flask import current_app, Flask, redirect, request, session, url_for,render_template
from flask_session import Session
import random    # 自定義,數學出題庫
import datetime
import guessnumlib as lib

def html_template( title, TE, SID):  
    return """
    <form method=POST>
    <div>{}</div>
    <div>{}</div>
    <input name=Ans>
    <input type=hidden name=SID value='{}'>
    <div>Try {}</div>
    <input type=submit>
    </form>
    """.format(TE["St"],TE["Tip"],SID,TE["TryCnt"])
     
def create_app():
    app = Flask(__name__)
    app.secret_key = 'secret'        
    app.config['SESSION_TYPE'] = 'filesystem'  # session类型为redis, filesystem
    app.config['SESSION_USE_SIGNER'] = True  # 是否对发送到浏览器上session的cookie值进行加密
    app.config['SESSION_PERMANENT'] = True  # 如果设置为True，则关闭浏览器session就失效。
    Session(app)

    # 根路由
    @app.route("/")
    def index():
        return redirect('/apps')

    @app.route("/apps")
    def list():
        books=["PF101.10.1-10","PF102.10.1-100","PF103.10.1-1000"]
        return "<ul>{}</ul>".format(
            "".join(f"<li><a href=apps/{r}>{r}</a></li>" for r in books))

    @app.route("/apps/<QID>", methods=['GET', 'POST'])
    def MathPanel(QID):
        QIID=QID.split(".")[0]
        if request.method == 'POST':
            SID = request.form["SID"]
            TE=session.get(SID,None)
            TE["TryCnt"]+=1
            TE["Ans"]=request.form["Ans"]
            lib.Post_Expr_CheckAns(TE)          
            if TE["OK"]==0:
                session[SID] =TE 
                return html_template(QID,TE,SID)
            else:
                session.pop(SID,None)
                return "<p>{}</p><a href=/>返回</a>".format(TE["Tip"])

        TE=lib.Get_Expr(QIID)
        SID=lib.GetKey()
        session[SID] =TE        
        return html_template(QID,TE,SID)

    # 容錯處理
    @app.errorhandler(500)
    def server_error(e):
        return """內部錯誤: <pre>{}</pre>,請檢查日誌.""".format(e), 500

    return app

app = create_app()

if __name__ == '__main__':
    app.run( host="0.0.0.0",port=8881, debug=True)

## 1.6 使用Session 模板: guessnum_template.py 

In [ ]:
from flask import current_app, Flask, redirect, request, session, url_for,render_template
from flask_session import Session
import random    # 自定義,數學出題庫
import datetime
import guessnumlib as lib

def create_app():
    app = Flask(__name__)
    app.secret_key = 'secret'        
    app.config['SESSION_TYPE'] = 'filesystem'  # session类型为redis, filesystem
    app.config['SESSION_USE_SIGNER'] = True  # 是否对发送到浏览器上session的cookie值进行加密
    app.config['SESSION_PERMANENT'] = True  # 如果设置为True，则关闭浏览器session就失效。
    Session(app)

    # 根路由
    @app.route("/")
    def index():
        return redirect('/apps')

    # 使用模版list.html, 顯示列表 
    @app.route("/apps")
    def list():
        books=["PF101.10.1-10","PF102.10.1-100","PF103.10.1-1000"]
        return render_template(
            "list.html",
            books=books
        )        

    @app.route("/apps/<QID>", methods=['GET', 'POST'])
    def MathPanel(QID):
        QIID=QID.split(".")[0]
        if request.method == 'POST':
            SID = request.form["SID"]
            TE=session.get(SID,None)
            TE["TryCnt"]+=1
            TE["Ans"]=request.form["Ans"]
            lib.Post_Expr_CheckAns(TE)          
            if TE["OK"]==0:
                session[SID] =TE 
            else:
                session.pop(SID,None)
            return render_template("guessnum.html", title=QID, TE=TE, SID=SID)                

        TE=lib.Get_Expr(QIID)
        SID=lib.GetKey()
        session[SID] =TE        
        return render_template("guessnum.html", title=QID, TE=TE, SID=SID)                                

    # 容錯處理
    @app.errorhandler(500)
    def server_error(e):
        return """內部錯誤: <pre>{}</pre>,請檢查日誌.""".format(e), 500

    return app

app = create_app()

# the application.
if __name__ == '__main__':
    app.run( host="0.0.0.0",port=8881, debug=True)

## 1.7 Flask-SQLAlchemy 2.2.4

## 1.8 Redis

## 1.9 用戶管理

# 參考資料

## 數學網站實作
python -m venv env     
env\scripts\activate  
pip install -r requirements.txt  
python main.py  
env\scripts\deactivate  
requirements.txt  

```
Flask==1.1.2
Flask-Session==0.3.2
six==1.15.0
httplib2==0.18.1
sympy==1.7.1
```
檔案結構   
esmathlib.py  
main.py  
templates\base.html   
templates\list.html  
templates\guessnum.html 

## main.py


In [ ]:
from flask import current_app, Flask, redirect, request, session, url_for,render_template
from flask_session import Session
import esmathlib     # 自定義,數學出題庫

def create_app():
   
    app = Flask(__name__)
    app.secret_key = 'secret'        
    app.config['SESSION_TYPE'] = 'filesystem'  # session类型为redis, filesystem
    app.config['SESSION_USE_SIGNER'] = True  # 是否对发送到浏览器上session的cookie值进行加密
    app.config['SESSION_PERMANENT'] = True  # 如果设置为True，则关闭浏览器session就失效。
    Session(app)

    # 根路由
    @app.route("/")
    def index():
        return redirect('/apps')

    # 使用模版list.html, 顯示列表 
    @app.route("/apps")
    def list():
        books=["PF101.有理數運算","PF102.整數指數冪運算","PF103.一元一次方程","PF104.整式的加減法練習","PF105.二元一次方程","PF106.一元一次不等式","PF107.一元一次不等式組","PF108.整式的乘法練習","PF201.根式的運算","PF202.整式的乘法公式平方差","PF203.因式分解提公因式","PF204.分式的乘除","PF205.分式的加減","PF206.分式方程","PF207.一次函數圖像的性質","PF291.一元二次方程式十字相乘法求因式","PF292.一元二次方程式求解","PF293.整式的乘法練習","PF301.一元二次方程式","PF302.解可化為一元二次方程的分式方程","PF303.解二元二次方程組","PF304.二次函數圖像的性質","PF305.解直角三角形"]
        return render_template(
            "list.html",
            books=books
            )

    # GET 顯示4題QID相關算式
    # POST 收集作答,並對比答案.
    @app.route("/apps/<QID>", methods=['GET', 'POST'])
    def MathPanel(QID):
        QIID=QID.split(".")[0]
        if request.method == 'POST':
            # POST 收集作答,並對比答案.
            SessKey = request.form["SID"]
            NTE=session.get(SessKey,None)
            del session[SessKey]    
            # Update and Combine NTE and Ans
            esmathlib.Post_Expr_UpdateAns(request.form,NTE) 
            esmathlib.Post_Expr_CheckAns(QIID,NTE)          
            return render_template("result.html", title=QID, NTE=NTE)    

        # GET 顯示4題QID相關算式
        NTE=esmathlib.Get_Expr(QIID)
        if NTE==None: return "None"
        SessKey=NTE[0]['Key']
        session[SessKey] =NTE        
        return render_template("form.html", title=QID, NTE=NTE, sid=SessKey)            

    # 容錯處理
    @app.errorhandler(500)
    def server_error(e):
        return """內部錯誤: <pre>{}</pre>,請檢查日誌.""".format(e), 500

    return app

app = create_app()

# the application.
if __name__ == '__main__':
    app.run( host="0.0.0.0",port=8081, debug=True)


## templates/base.html

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <title>Python Math </title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="//maxcdn.bootstrapcdn.com/bootstrap/3.3.2/css/bootstrap.min.css">
    <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
    <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"></script>
  </head>
  <body>
    <div class="navbar navbar-default">
      <div class="container">
        <div class="navbar-header">
          <div class="navbar-brand">Math</div>
        </div>
        <ul class="nav navbar-nav">
          <li><a href="/"><span class="glyphicon">&#xe021</span></a></li>
          <li><a href="/apps">List</a></li>
        </ul>
      </div>
    </div>
    <div class="container">
      {% block content %}{% endblock %}
    </div>
    {{user}}
  </body>
</html>
```

## templates/list.html

```html
{% extends "base.html" %}

{% block content %}

<h3>List</h3>

{% for book in books %}
<div class="media">
  <a href="/apps/{{book}}">
      <h4>{{book}}</h4>
  </a>
</div>
{% else %}
<p>No books found</p>
{% endfor %}


{% endblock %}
```

## templates/guessnum.html

```html
{% extends "base.html" %}

{% block content %}


<h3>{{title}}</h3>
<div style="margin: auto;width: 30%;">
  <form method="POST">
    <p>St:{{TE.St}}</p>  
    <p>Tip:{{TE.Tip}}</p>  
    <p>try times:{{TE.TryCnt}}</p>  
    <P>
      <input name="SID" type="hidden" value="{{SID}}">
    </P>
    {% if TE.OK==0 %}
    <P>
        <input name="Ans" >
        <div><input class="btn btn-success" type="submit" value="提交試題"></div>
    </P>
    {% else %}
    
    {% endif %}

  </form>
</div>
{% endblock %}
```

templates/form.html
```html
{% extends "base.html" %}

{% block content %}
<h3>{{title}}</h3>
<div style="margin: auto;width: 30%;">
<form method="POST">
{% for TE in NTE %}
    {% if TE.St is iterable and TE.St is not string  %}
    <div>題目:</div>
    <div> \[{{TE.St|safe}}\]</div>
    <div>{{TE.Tip}}</div>
    <div>作答:<input name="{{TE.Id}}" autocomplete="off"></div>
    <input name="SID" type="hidden" value="{{sid}}">
{% else %}
    <p>No books found</p>
{% endfor %}
<div><input type="submit"></div>
</form>
</div>
{% endblock %}

```
templates/result.html
```html
{% extends "base.html" %}

{% block content %}
<style>
    td {padding-right:30px}
</style>
<h3>{{title}}</h3>
<div style="  margin: auto;width: 80%;">
    <table>
        <tr>
            <td>St</td>
            <td>Val</td>
            <td>Ans</td>
            <td>OK</td>
        </tr>
        {% for TE in NTE %}
        <tr>
            <td>
                <div> \[{{TE.St|safe}}\]</div>
            </td>
            <td>
                <div>{{TE.Val}}</div>
            </td>
            <td>
                <div>{{TE.Ans}}</div>
            </td>
            <td>
                <div>{{TE.OK}}</div>
            </td>            
        </tr>
        {% endfor %}
    </table>
    <a href="/apps" class="btn btn-success btn-sm">
        Return
      </a>
</div>
{% endblock %}
```